In [52]:
import pandas as pd 
import re

state_populations_2023 = { # use csv census data instead
    "Alabama": 5074296,
    "Alaska": 733391,
    "Arizona": 7359197,
    "Arkansas": 3045637,
    "California": 39538223,
    "Colorado": 5773714,
    "Connecticut": 3605597,
    "Delaware": 989948,
    "Florida": 21944577,
    "Georgia": 10711908,
    "Hawaii": 1455271,
    "Idaho": 1839106,
    "Illinois": 12812508,
    "Indiana": 6833037,
    "Iowa": 3219171,
    "Kansas": 2937150,
    "Kentucky": 4505836,
    "Louisiana": 4624047,
    "Maine": 1372247,
    "Maryland": 6185278,
    "Massachusetts": 7033469,
    "Michigan": 10077331,
    "Minnesota": 5706494,
    "Mississippi": 2961279,
    "Missouri": 6168187,
    "Montana": 1104271,
    "Nebraska": 1961504,
    "Nevada": 3104614,
    "New Hampshire": 1377529,
    "New Jersey": 9288994,
    "New Mexico": 2117522,
    "New York": 20201249,
    "North Carolina": 10551162,
    "North Dakota": 779094,
    "Ohio": 11780017,
    "Oklahoma": 3990443,
    "Oregon": 4266560,
    "Pennsylvania": 12964056,
    "Rhode Island": 1097379,
    "South Carolina": 5282634,
    "South Dakota": 909824,
    "Tennessee": 7051339,
    "Texas": 31000000,
    "Utah": 3271616,
    "Vermont": 643077,
    "Virginia": 8654542,
    "Washington": 7738692,
    "West Virginia": 1793716,
    "Wisconsin": 5893718,
    "Wyoming": 576851,
    "District of Columbia": 689545,
    "Puerto Rico": 3263584
}



df = pd.DataFrame(pd.read_excel("../data/outages/2023_Annual_Summary.xls")) # can also obtain thru api: https://openenergyhub.ornl.gov/explore/dataset/oe-417-annual-summaries/api/
  
df.columns = df.iloc[0]
df = df.drop(index=0)

df = df[df["Number of Customers Affected"] != 0]
df = df[df["Number of Customers Affected"] != "0"]
df = df[df["Number of Customers Affected"] != "Unknown"]

for _, row in df.iterrows():
    row["Area Affected"] = re.sub(r":.*", "", row["Area Affected"]) # need to handle multiple states...

    if row["Area Affected"] not in state_populations_2023.keys():
        if row["Area Affected"] == "LUMA Energy":
            row["Area Affected"] = "Puerto Rico"
        if row["Area Affected"] == "ISO New England":
            row["Area Affected"] = "Connecticut" #, Maine, Massachusetts, New Hampshire, Rhode Island, Vermont"
        if row["Area Affected"] == "Otter Tail Power Co":
            row["Area Affected"] = "Minnesota" #, North Dakota, South Dakota"
        if "Western Area Power" in row["Area Affected"]:
            row["Area Affected"] = "Montana" #, North Dakota, South Dakota, Nebraska, Iowa, Minnesota"

df

,Event Month,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Area Affected,NERC Region,Alert Criteria,Event Type,Demand Loss (MW),Number of Customers Affected
3,January,01/01/2023,14:47:00,01/01/2023,16:50:00,Florida,SERC,Physical attack that could potentially impact ...,Vandalism,19,3509
6,January,01/02/2023,18:22:00,01/02/2023,22:27:00,Arkansas,SERC,"Unexpected Transmission loss within its area, ...",Severe Weather,20,6849
8,January,01/05/2023,08:19:00,01/05/2023,08:20:00,Louisiana,SERC,Electrical System Separation (Islanding) where...,Transmission Interruption,23,1631
11,January,01/07/2023,22:00:00,01/08/2023,21:00:00,California,WECC,"Loss of electric service to more than 50,000 c...",Severe Weather,458,185434
12,January,01/07/2023,23:44:00,01/08/2023,02:29:00,Texas,TRE,"Unexpected Transmission loss within its area, ...",Transmission Interruption,20,9823
...,...,...,...,...,...,...,...,...,...,...,...
337,December,2023-12-18 00:00:00,05:45:00,- Unknown,- Unknown,Connecticut,WECC,"Loss of electric service to more than 50,000 c...",- Weather or natural disaster,Unknown,730000
339,December,2023-12-19 00:00:00,00:41:00,2023-12-19 00:00:00,00:47:00,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,77,50204
341,December,2023-12-23 00:00:00,00:43:00,2023-12-23 00:00:00,00:48:00,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,65,37850
343,December,2023-12-25 00:00:00,19:56:00,2023-12-25 00:00:00,20:01:00,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,105,36100


In [55]:
from datetime import datetime, timedelta

outages = []
dic = {}
for _, row in df.iterrows():
    # if row["Date Event Began"] == row["Date of Restoration"]:
    #     starttime = row["Time Event Began"]
    #     endtime = row["Time of Restoration"]
    #     diff = endtime - starttime
    #     lst.append(str(diff))

    try: 
        start = row["Date Event Began"] + " " + str(row["Time Event Began"])
        end = row["Date of Restoration"] + " " + str(row["Time of Restoration"])
    except:
        start = "error"
        end = "error"

    #print(type(row["Date Event Began"]), type(row["Time Event Began"]))

    state = row["Area Affected"]
    if row["Number of Customers Affected"] == "Unknown":
        continue
    if state not in dic:
        dic[state] = int(row["Number of Customers Affected"])
    else:
        dic[state] += int(row["Number of Customers Affected"])

    outages.append((row["Area Affected"], start, end))


# del dic["Arkansas, Mississippi"] # get rid of this eventually
# del dic["Wisconsin, Michigan"]

dic

# dic = {x: round((y/state_populations_2023[x])*100, 2) for x,y in dic.items()} # handle multiple states

# print("(ROUGH) Percent of state residents affected by a power outage in 2023:")
# dic



# df.plot(x='Demand Loss (MW)', y='Number of Customers Affected', kind='line')
# plt.show()


{'Florida': 6056,
 'Arkansas': 200601,
 'Louisiana': 31286,
 'California': 348077,
 'Texas': 1759778,
 'Alabama': 210384,
 'Connecticut': 1097000,
 'Michigan': 1906580,
 'Wisconsin': 327000,
 'Mississippi': 77576,
 'Kentucky': 346000,
 'Ohio': 335987,
 'Tennessee': 667000,
 'Nevada': 11479,
 'Minnesota': 88000,
 'Illinois': 310842,
 'South Carolina': 8,
 'South Dakota': 1,
 'North Dakota': 656,
 'Wisconsin, Michigan': 70000,
 'Arkansas, Mississippi': 64732,
 'Indiana': 140000,
 'Missouri': 470658,
 'New York': 1284,
 'Georgia': 70514,
 'West Virginia': 104196,
 'Maryland': 337576,
 'Montana': 1,
 'Puerto Rico': 778582}

In [46]:
outages

[('Florida', '01/01/2023 14:47:00', '01/01/2023 16:50:00'),
 ('Arkansas', '01/02/2023 18:22:00', '01/02/2023 22:27:00'),
 ('Louisiana', '01/05/2023 08:19:00', '01/05/2023 08:20:00'),
 ('California', '01/07/2023 22:00:00', '01/08/2023 21:00:00'),
 ('Texas', '01/07/2023 23:44:00', '01/08/2023 02:29:00'),
 ('Alabama', '01/12/2023 14:00:00', '01/13/2023 03:00:00'),
 ('Connecticut', '01/23/2023 07:05:00', '01/24/2023 17:17:00'),
 ('Texas', '01/24/2023 15:25:00', '01/26/2023 20:20:00'),
 ('Arkansas', '01/25/2023 03:30:00', '01/25/2023 13:00:00'),
 ('Texas', '01/31/2023 08:15:00', '02/06/2023 17:00:00'),
 ('Texas', '02/01/2023 05:00:00', '02/04/2023 22:39:00'),
 ('Arkansas', '02/02/2023 08:15:00', '02/02/2023 19:45:00'),
 ('Florida', '02/21/2023 13:14:00', '02/21/2023 14:36:00'),
 ('Michigan', '02/22/2023 18:41:00', '02/25/2023 16:12:00'),
 ('Michigan', '02/22/2023 18:30:00', 'Unknown Unknown'),
 ('Wisconsin', '02/23/2023 04:30:00', '02/23/2023 13:00:00'),
 ('California', '02/25/2023 00:08:00

In [10]:
# datetime tutorial

from datetime import datetime

datetime_str = '13:55:26'

datetime_object = datetime.strptime(datetime_str, '%H:%M:%S')
time_object = datetime.strptime(datetime_str, '%H:%M:%S').time() #stripping date (.date() strips the time)

print(type(datetime_object), type(time_object))
print("date and time:", datetime_object)
print("only time:", time_object)

<class 'datetime.datetime'> <class 'datetime.time'>
date and time: 1900-01-01 13:55:26
only time: 13:55:26


In [1]:
def clean_outages(path):

    df = pd.DataFrame(pd.read_excel(path))
  
    df.columns = df.iloc[0]
    df = df.drop(index=0)

    df = df[df["Number of Customers Affected"] != 0]
    df = df[df["Number of Customers Affected"] != "0"]
    df = df[df["Number of Customers Affected"] != "Unknown"]

    for _, row in df.iterrows():
        row["Area Affected"] = "".join(re.findall(r"\b\w+:", row["Area Affected"])) # need to handle multiple states...

        if row["Area Affected"] not in state_pops_23.keys():
            if row["Area Affected"] == "LUMA Energy":
                row["Area Affected"] = "Puerto Rico"
            if row["Area Affected"] == "ISO New England":
                row["Area Affected"] = "Connecticut" #, Maine, Massachusetts, New Hampshire, Rhode Island, Vermont"
            if row["Area Affected"] == "Otter Tail Power Co":
                row["Area Affected"] = "Minnesota" #, North Dakota, South Dakota"
            if "Western Area Power" in row["Area Affected"]:
                row["Area Affected"] = "Montana" #, North Dakota, South Dakota, Nebraska, Iowa, Minnesota"
                
    return df


def build_outage_dict(path):

    df = clean_outages(path)

    dic = {}
    for _, row in df.iterrows():

        num_affected = row["Number of Customers Affected"]

        state = row["Area Affected"]
        if row["Area Affected"] not in dic:
            dic[state] = int(num_affected)
        else:
            dic[state] += int(num_affected)

    dic2 = {}
    for state, count in dic.items():
        states = state.split(":")
        for state in states:
            if state not in dic2:
                dic2[state] = count/len(states)
            else:
                dic2[state] += count/len(states)
    # del dic["Arkansas, Mississippi"] # get rid of this eventually
    # del dic["Wisconsin, Michigan"]
    # del dic["Washington, Idaho, Montana"]
    # del dic['Pacificorp']

    return dic2

    return {x: round((y/state_pops_23[x])*100, 2) for x,y in dic.items()} # handle multiple states

